In [2]:
import tensorflow as tf

In [3]:
sess = tf.Session()

# tf.nn.sigmoid_cross_entropy_with_logits

In [41]:
x = [[1.,3.,4.],[2.,5.,6.]]
print("x")
print(x)
x_true = [[1.,0.,0.],[0.,1.,0.]]
print("x_true")
print(x_true)
x_sigmoid = tf.sigmoid(x)
print("tf.sigmoid(x)")
print(sess.run(x_sigmoid))
print("First method")
ones = tf.ones_like(x_sigmoid)
print(-sess.run(x_true*tf.log(x_sigmoid)+(ones-x_true)*tf.log(ones-x_sigmoid)))
print("Second method")
print(sess.run(tf.nn.sigmoid_cross_entropy_with_logits(labels = x_true, logits= x)))

x
[[1.0, 3.0, 4.0], [2.0, 5.0, 6.0]]
x_true
[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]]
tf.sigmoid(x)
[[ 0.7310586   0.95257413  0.98201376]
 [ 0.88079703  0.99330717  0.99752742]]
First method
[[ 0.31326166  3.04858756  4.01814842]
 [ 2.12692761  0.00671532  6.00249338]]
Second method
[[ 0.31326169  3.04858732  4.01814985]
 [ 2.12692809  0.00671535  6.00247574]]


# normalize with tf.reduce_mean

In [22]:
x = tf.constant([[1,2,3],[4,5,6]])
y = tf.constant([2,2,2])
print(sess.run(x-y))

[[-1  0  1]
 [ 2  3  4]]


In [24]:
x = tf.constant([[1,2,3],[4,5,6]], dtype = tf.float32)
print("x : {}".format(sess.run(x).shape))
mean = tf.reduce_mean(x,[0])
print("mean = tf.reduce_mean(x,[0])")
print(sess.run(mean))
print("x_mean")
print(sess.run(x-mean))
std =tf.reduce_mean(tf.square(x-mean),[0])
print("std =tf.reduce_mean(tf.square(x-mean),[0])")
print(sess.run(std))

x : (2, 3)
mean = tf.reduce_mean(x,[0])
[ 2.5  3.5  4.5]
x_mean
[[-1.5 -1.5 -1.5]
 [ 1.5  1.5  1.5]]
std =tf.reduce_mean(tf.square(x-mean),[0])
[ 2.25  2.25  2.25]


# tf.get_shape()

In [15]:
x = tf.constant([[1,2,3],[4,5,6]])
print("x : {}".format(x.get_shape()))
print(sess.run(x))
print("x.get_shape().ndims")
print(x.get_shape().ndims)
print("tf.expand_dims(x, dim=1).get_shape().ndims")
print(tf.expand_dims(x, dim=1).get_shape().ndims)

x : (2, 3)
[[1 2 3]
 [4 5 6]]
x.get_shape().ndims
2
tf.expand_dims(x, dim=1).get_shape().ndims
3


# tf.tile()

In [8]:
x = tf.constant([[1,2,3],[4,5,6]])
print("x : {}".format(sess.run(x).shape))
print(sess.run(x))
print("tf.tile(x,[2,3]): {}".format(sess.run(tf.tile(x,[2,3])).shape))
print(sess.run(tf.tile(x,[2,3])))

x : (2, 3)
[[1 2 3]
 [4 5 6]]
tf.tile(x,[2,3]): (4, 9)
[[1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]
 [1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]]


# tf.expand_dims

In [10]:
x = tf.constant([[1,2,3],[4,5,6]])
print("x : {}".format(sess.run(x).shape))
print(sess.run(x))
print("tf.expand_dims(x, dim=1) : {}".format(sess.run(tf.expand_dims(x, dim=1)).shape))
print(sess.run(tf.expand_dims(x, dim=1)))
print("tf.expand_dims(x, dim=0) : {}".format(sess.run(tf.expand_dims(x, dim=0)).shape))
print(sess.run(tf.expand_dims(x, dim=0)))
print("tf.expand_dims(x, dim=2) : {}".format(sess.run(tf.expand_dims(x, dim=2)).shape))
print(sess.run(tf.expand_dims(x, dim=2)))
print("tf.expand_dims(x, dim=-1) : {}".format(sess.run(tf.expand_dims(x, dim=-1)).shape))
print(sess.run(tf.expand_dims(x, dim=-1)))
'''
print("tf.expand_dims(x, dim=3) : {}".format(sess.run(tf.expand_dims(x, dim=3)).shape))
print(sess.run(tf.expand_dims(x, dim=3)))
Error occured
'''


x : (2, 3)
[[1 2 3]
 [4 5 6]]
tf.expand_dims(x, dim=1) : (2, 1, 3)
[[[1 2 3]]

 [[4 5 6]]]
tf.expand_dims(x, dim=0) : (1, 2, 3)
[[[1 2 3]
  [4 5 6]]]
tf.expand_dims(x, dim=2) : (2, 3, 1)
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]]
tf.expand_dims(x, dim=-1) : (2, 3, 1)
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]]


'\nprint("tf.expand_dims(x, dim=3) : {}".format(sess.run(tf.expand_dims(x, dim=3)).shape))\nprint(sess.run(tf.expand_dims(x, dim=3)))\nError occured\n'

# tf.stack, tf.unstack, tf.split

In [11]:
v1 = tf.constant([1,2,3])
v2 = tf.constant([4,5,6])
t = tf.stack([v1,v2])
print(sess.run(t))
print(sess.run(tf.unstack(t)))
print(sess.run(tf.split(t, 2, axis=0)))

[[1 2 3]
 [4 5 6]]
[array([1, 2, 3], dtype=int32), array([4, 5, 6], dtype=int32)]
[array([[1, 2, 3]], dtype=int32), array([[4, 5, 6]], dtype=int32)]


# tf.get_variable() v.s tf.variable

In [3]:
v = tf.Variable(tf.random_normal([2,3]), name='v')
x = tf.get_variable('v', [2,3],initializer= tf.random_normal_initializer())
y = tf.get_variable('y', [3,3])
sess.run(tf.global_variables_initializer())
print(sess.run(v))
print(sess.run(x))
print(sess.run(y))

[[-0.95063412 -0.29145828 -0.10211317]
 [-0.81984377  0.83733428  0.06603181]]
[[-1.50503135 -1.15622473  0.42767996]
 [ 0.11276423 -1.15868485  0.90092182]]
[[-0.12855768  0.00583577 -0.8714211 ]
 [ 0.24863982  0.35162139 -0.72656345]
 [-0.61718869 -0.48462391 -0.75277734]]


 # Share variables with scope

In [4]:
with tf.variable_scope('hi') as scope:
    v = tf.get_variable('a', [2,3],initializer= tf.random_normal_initializer())
    scope.reuse_variables()
    x = tf.get_variable('a', [2,3],initializer= tf.random_normal_initializer())
    sess.run(tf.global_variables_initializer())
    print(sess.run(v))
    print(sess.run(x))
    

[[-0.30628705 -0.39645576 -0.4932133 ]
 [ 1.5868845  -0.87908548  0.93908131]]
[[-0.30628705 -0.39645576 -0.4932133 ]
 [ 1.5868845  -0.87908548  0.93908131]]


# tf.split()

In [3]:
value = tf.random_normal([5,30],stddev=0.1)
print(sess.run(tf.shape(value)))

split0, split1, split2 = tf.split(value, [4, 15, 11], 1)
print(sess.run(tf.shape(split0)))
print(sess.run(tf.shape(split1)))
print(sess.run(tf.shape(split2)))
split0, split1, split2 = tf.split(value, 3, axis=1)
print(sess.run(tf.shape(split0)))
splits = tf.split(value, 3, axis=1)
print(sess.run(tf.shape(splits)))
splits = tf.split(value, 5, axis=0)
print(sess.run(tf.shape(splits)))

[ 5 30]
[5 4]
[ 5 15]
[ 5 11]
[ 5 10]
[ 3  5 10]
[ 5  1 30]
